<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9818e475f6c4a60f2191be1b7df4f2d96a9d76e307f3720f12ff234eae9bc8a9
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 13:46:23
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.41 C
-------------------
Today PnL: -1.03 L (-0.72%)
Current PnL: -24.42 L (-16.0%)
CY Booked + Current PnL: -10.53 L (-6.9%)
-------------------
Total profit:  1.46 L
Total loss:  -25.88 L
-------------------
Total Booked + Current PnL: 16.45 L (13.09%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.83%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.83 L (63.54%)
Deployed:  1.26 C
Current:  1.41 C
CAGR/XIRR %: 6.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.19,-9.31,10.28,0.02,16274.0,-16248.0,158311.0,141.63,65.0,M-SC,14.22,234.0,-1.00,1.12,37.70,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.12,-16.65,20.09,0.09,16875.0,-16780.0,83997.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.59,10.35,OX40N,NTT,DURABLES
51,MASFIN,398.61,0.24,-2.33,24.96,22.05,23887.0,-2280.0,95700.0,-15.99,59.0,H-SC,6.72,164.0,-0.10,0.68,39.11,XR,ATH,FINANCE
43,ITC,452.00,-0.21,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.20,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.85,5.09,14.37,20.19,26326.0,8881.0,183201.0,-17.67,45.0,X-MC,6.72,68.0,0.34,1.30,20.98,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,5.83,-46.33,214.96,69.05,153529.0,-61644.0,71422.0,-56.49,55.0,H-SC,23.75,140.0,-0.40,0.51,10.15,SR,ATH,IT
13,BSOFT,831.70,3.78,-23.81,97.91,50.79,104924.0,-33489.0,107164.0,-2.20,71.0,H-SC,5.89,151.0,-0.32,0.76,24.77,XR,ATH,IT
85,WIPRO,420.00,1.99,4.65,64.61,72.27,114572.0,7881.0,177329.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.11,XR,NTT,IT
72,TANLA,1943.92,1.43,-36.25,244.61,119.69,422234.0,-98146.0,172615.0,-35.52,36.0,H-SC,8.63,156.0,-0.23,1.22,36.36,AR,ATH,IT
16,CERA,9475.00,1.11,-28.48,73.23,23.89,92122.0,-50105.0,125798.0,-31.03,33.0,H-SC,7.27,157.0,-0.54,0.89,8.64,OX40N,NTT,CERAMICS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,-5.05,-39.30,86.01,12.90,114067.0,-85875.0,132621.0,95.91,46.0,H-SC,12.17,161.0,-0.75,0.94,10.27,OX40N,NTT,FINANCE
24,EASEMYTRIP,26.40,-4.22,-54.86,222.74,45.70,177791.0,-96995.0,79820.0,-17.09,60.0,M-SC,23.44,205.0,-0.55,0.56,15.05,XY24,NTT,TRAVEL
18,COFFEEDAY,80.00,-2.95,-42.63,135.78,35.27,88455.0,-48403.0,65146.0,-55.73,27.0,L-SC,14.77,270.0,-0.55,0.46,59.45,XR,NTT,HOTELS
22,DIXON,18931.72,-2.86,-9.58,34.46,21.58,43668.0,-13419.0,126720.0,-54.37,29.0,X-MC,5.86,56.0,-0.31,0.90,12.19,X40N,ATH,IT
84,WHIRLPOOL,2270.00,-2.70,-21.25,131.02,81.92,118441.0,-24399.0,90399.0,-53.88,22.0,M-SC,4.96,236.0,-0.21,0.64,7.02,XR,NTT,DURABLES


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.13,0.09,106.54,106.72,131521.0,112.0,123448.0,-50.44,40.0,H-SC,1.72,139.0,0.00,0.87,23.49,AR,ATH,MISC
37,IEX,219.00,-0.42,0.39,48.05,48.63,96449.0,773.0,200727.0,-33.16,64.0,H-SC,14.79,136.0,0.01,1.42,11.98,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.41,-6.05,121.29,107.91,168250.0,-8927.0,138717.0,-23.39,32.0,M-SC,11.15,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.19,-9.31,10.28,0.02,16274.0,-16248.0,158311.0,141.63,65.0,M-SC,14.22,234.0,-1.00,1.12,37.70,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.12,-16.65,20.09,0.09,16875.0,-16780.0,83997.0,94.88,37.0,M-SC,2.93,253.0,-0.99,0.59,10.35,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.15,-23.14,47.63,13.47,98727.0,-62388.0,207279.0,-68.31,31.0,H-SC,2.15,158.0,-0.63,1.47,5.40,XY24,NTT,PAINTS
67,SIS,528.00,1.04,-24.50,60.61,21.26,50810.0,-27201.0,83831.0,1977.20,47.0,H-SC,3.91,166.0,-0.54,0.59,13.26,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.99,4.65,64.61,72.27,114572.0,7881.0,177329.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.11,XR,NTT,IT
37,IEX,219.00,-0.42,0.39,48.05,48.63,96449.0,773.0,200727.0,-33.16,64.0,H-SC,14.79,136.0,0.01,1.42,11.98,XR,NTT,MISC
38,INDIAMART,4810.62,0.13,0.09,106.54,106.72,131521.0,112.0,123448.0,-50.44,40.0,H-SC,1.72,139.0,0.00,0.87,23.49,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.99,4.65,64.61,72.27,114572.0,7881.0,177329.0,-10.16,67.0,M-LC,3.12,99.0,0.07,1.25,11.11,XR,NTT,IT
37,IEX,219.00,-0.42,0.39,48.05,48.63,96449.0,773.0,200727.0,-33.16,64.0,H-SC,14.79,136.0,0.01,1.42,11.98,XR,NTT,MISC
38,INDIAMART,4810.62,0.13,0.09,106.54,106.72,131521.0,112.0,123448.0,-50.44,40.0,H-SC,1.72,139.0,0.00,0.87,23.49,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.63,-1.21,37.29,35.63,77203.0,-2541.0,207033.0,-14.97,43.0,H-MC,3.63,119.0,-0.03,1.46,15.26,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.41,-6.05,121.29,107.91,168250.0,-8927.0,138717.0,-23.39,32.0,M-SC,11.15,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,0.29,-20.42,86.54,48.45,138094.0,-40934.0,159572.0,-26.10,15.0,X-MC,8.84,64.0,-0.30,1.13,0.29,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,0.03,-37.75,117.20,35.20,93871.0,-48575.0,80095.0,4.98,18.0,X-SC,14.25,92.0,-0.52,0.57,0.03,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-1.61,-9.61,39.47,26.07,58416.0,-15740.0,148000.0,-33.39,27.0,X-MC,10.52,55.0,-0.27,1.05,0.00,X40,ATH,APPARELS
8,AWL,485.00,-0.64,-21.57,95.72,53.50,226520.0,-65102.0,236649.0,-62.10,28.0,X-MC,1.97,58.0,-0.29,1.67,5.11,XY24,NTT,FMCG
23,DMART,5391.45,-1.56,-7.32,38.25,28.12,74586.0,-15412.0,194995.0,-20.03,28.0,X-LC,4.36,19.0,-0.21,1.38,14.57,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,-0.64,-22.67,112.28,64.15,206595.0,-53951.0,184000.0,-54.57,46.0,X-SC,1.45,82.0,-0.26,1.30,3.21,XY24,BTT,CEMENT
43,ITC,452.0,-0.21,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.20,X40,NTT,FMCG
15,CAMS,4762.0,0.32,1.24,23.26,24.79,59308.0,3122.0,254978.0,-3.47,47.0,X-SC,1.92,86.0,0.05,1.80,24.67,X40N,NTT,MISC
8,AWL,485.0,-0.64,-21.57,95.72,53.50,226520.0,-65102.0,236649.0,-62.10,28.0,X-MC,1.97,58.0,-0.29,1.67,5.11,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-1.05,1.14,29.50,30.97,52730.0,2007.0,178745.0,-22.64,47.0,X-MC,2.17,75.0,0.04,1.26,13.84,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COLPAL,3726.84,-2.33,-19.96,76.80,41.51,161887.0,-52575.0,210790.0,-4.76,31.0,X-MC,7.31,61.0,-0.32,1.49,0.00,XY25,ATH,FMCG
54,PAGEIND,51605.08,-1.61,-9.61,39.47,26.07,58416.0,-15740.0,148000.0,-33.39,27.0,X-MC,10.52,55.0,-0.27,1.05,0.00,X40,ATH,APPARELS
56,QUESS,424.00,-0.34,-29.70,103.20,42.84,47159.0,-19309.0,45697.0,-53.72,38.0,X-SC,37.41,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
55,PGHH,17907.65,-0.51,-5.62,41.73,33.76,79089.0,-11295.0,189525.0,-33.45,29.0,X-MC,4.52,57.0,-0.14,1.34,0.00,X40,ATH,FMCG
11,BATAINDIA,2096.00,0.03,-37.75,117.20,35.20,93871.0,-48575.0,80095.0,4.98,18.0,X-SC,14.25,92.0,-0.52,0.57,0.03,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.34,-29.70,103.20,42.84,47159.0,-19309.0,45697.0,-53.72,38.0,X-SC,37.41,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.67,-47.74,191.41,52.31,145217.0,-69293.0,75867.0,-44.34,34.0,X-SC,5.04,91.0,-0.48,0.54,1.55,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.03,-37.75,117.20,35.20,93871.0,-48575.0,80095.0,4.98,18.0,X-SC,14.25,92.0,-0.52,0.57,0.03,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-2.86,-9.58,34.46,21.58,43668.0,-13419.0,126720.0,-54.37,29.0,X-MC,5.86,56.0,-0.31,0.90,12.19,X40N,ATH,IT
52,MEDANTA,1486.00,-1.60,0.61,23.32,24.08,29787.0,780.0,127730.0,-7.57,39.0,X-SC,5.83,89.0,0.03,0.90,20.35,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.07,-14.80,40.00,19.28,117902.0,-51200.0,294756.0,-25.89,58.0,X-LC,6.92,1.0,-0.43,2.09,8.97,X40,ATH,IT
41,INFY,2275.00,0.89,7.21,44.45,54.87,149109.0,22566.0,335454.0,-16.37,63.0,X-LC,2.71,2.0,0.15,2.37,16.09,X40,BTT,IT
76,TMPV,600.00,-1.76,-15.78,68.82,42.18,160204.0,-43623.0,232787.0,-24.30,31.0,X-LC,3.15,3.0,-0.27,1.65,0.84,XY24,NTT,AUTO
82,VBL,671.64,-0.36,-3.11,40.03,35.67,122499.0,-9825.0,306017.0,-14.67,60.0,X-LC,5.39,4.0,-0.08,2.16,10.31,X40N,ATH,FMCG
43,ITC,452.00,-0.21,-2.24,12.97,10.44,25376.0,-4489.0,195649.0,-41.56,39.0,X-LC,1.52,5.0,-0.18,1.38,3.20,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.06,-37.26,111.81,32.89,54160.0,-28768.0,48439.0,-698.49,60.0,L-MC,6.01,259.0,-0.53,0.34,33.40,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.89,-15.65,115.37,81.67,91293.0,-14679.0,79131.0,7111.11,42.0,L-SC,18.72,271.0,-0.16,0.56,53.24,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.46,-12.40,59.29,39.54,52543.0,-12539.0,88620.0,-31.43,63.0,M-SC,6.63,220.0,-0.24,0.63,24.96,AR,ATH,AUTO
51,MASFIN,398.61,0.24,-2.33,24.96,22.05,23887.0,-2280.0,95700.0,-15.99,59.0,H-SC,6.72,164.0,-0.10,0.68,39.11,XR,ATH,FINANCE
13,BSOFT,831.70,3.78,-23.81,97.91,50.79,104924.0,-33489.0,107164.0,-2.20,71.0,H-SC,5.89,151.0,-0.32,0.76,24.77,XR,ATH,IT


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.19,-9.31,10.28,0.02,16274.0,-16248.0,158311.0,141.63,65.0,M-SC,14.22,234.0,-1.00,1.12,37.70,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.66,2.59,17.90,20.95,46232.0,6512.0,258280.0,1.62,75.0,X-LC,17.35,31.0,0.14,1.83,39.14,X40,ATH,PAINTS
13,BSOFT,831.70,3.78,-23.81,97.91,50.79,104924.0,-33489.0,107164.0,-2.20,71.0,H-SC,5.89,151.0,-0.32,0.76,24.77,XR,ATH,IT
50,LTIM,7201.88,-0.08,12.60,16.93,31.66,46922.0,31004.0,277155.0,8.32,69.0,H-LC,14.93,43.0,0.66,1.96,53.78,X200,ATH,IT
66,SIEMENS,4671.50,-0.51,-10.15,39.70,25.51,66378.0,-18895.0,167200.0,2.62,66.0,H-LC,2.73,49.0,-0.28,1.18,23.21,AR,ATH,ELECTRICAL


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.05
1,25,44.81
2,50,75.60


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.39
MC    28.90
LC    25.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.54
X40      21.30
X40N     11.92
XR        9.72
AR        9.11
XY25      9.06
OX40N     7.59
X200      1.96
SR        1.01
MH        0.76
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.93
X-MC    22.45
X-LC    20.31
M-SC    11.90
X-SC     8.18
H-MC     4.75
H-LC     3.14
L-SC     1.38
M-MC     1.36
M-LC     1.25
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.72,-6.38,40.78
IT,14.14,-13.16,71.57
FINANCE,9.31,-17.83,67.61
MISC,7.44,-23.31,73.83
ELECTRICAL,6.02,-10.21,50.58
PAINTS,6.01,-8.75,24.90
INSURANCE,4.38,-2.09,38.07
PHARMA,3.88,-1.55,34.05
AUTO,3.37,-20.96,72.83


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3140515.0,21
XR,1289418.0,13
AR,1282030.0,10
X40,1027117.0,14
X40N,763062.0,9
OX40N,719842.0,10
XY25,360306.0,6
SR,277782.0,2
MH,76265.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3595406.0,25
M-SC,1376572.0,15
X-MC,1306936.0,16
X-LC,891946.0,11
X-SC,762921.0,8
H-MC,407041.0,3
L-SC,266538.0,3
M-LC,114572.0,1
H-LC,113300.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1254355.0      6
           AR         881864.0      5
M-SC       XY24       786396.0      6
H-SC       XR         772505.0      7
X-MC       X40        462532.0      7
           XY24       386842.0      3
X-LC       X40        383036.0      5
H-SC       OX40N      332635.0      4
M-SC       OX40N      300417.0      5
X-SC       X40N       297831.0      3
           XY24       283541.0      3
H-SC       SR         277782.0      2
X-MC       X40N       268146.0      4
X-LC       XY24       234802.0      2
H-MC       AR         212462.0      2
X-LC       X40N       197085.0      2
H-MC       XY24       194579.0      1
X-MC       XY25       189416.0      2
X-SC       X40        181549.0      2
L-SC       XR         179748.0      2
M-SC       XR         168433.0      2
           AR         121326.0      2
M-LC       XR         114572.0      1
L-SC       OX40N       86790.0      1
X-LC       XY25        77023.0      2
H-SC       MH          76265.0      1
H-LC       AR          66378.0      1
L-MC       XR          54160.0      1
M-MC       XY25        52469.0      1
H-LC       X200        46922.0      1
L-LC       XY25        41398.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
